In [1]:
import torch
from torch import nn, optim
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt

In [2]:
traning_data = datasets.FashionMNIST(
    root='Data',
    train=True,
    download=True,
    transform=ToTensor()
)
test_data = datasets.FashionMNIST(
    root='Data',
    train=False,
    download=True,
    transform=ToTensor()
)

100%|██████████████████████████████████████████████████████████████████████████████| 26.4M/26.4M [01:25<00:00, 308kB/s]
100%|██████████████████████████████████████████████████████████████████████████████| 29.5k/29.5k [00:00<00:00, 137kB/s]
100%|██████████████████████████████████████████████████████████████████████████████| 4.42M/4.42M [00:12<00:00, 348kB/s]
100%|█████████████████████████████████████████████████████████████████████████████████████| 5.15k/5.15k [00:00<?, ?B/s]


In [3]:
batch_size =64
train_dataloader = DataLoader(traning_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data,batch_size=batch_size)

In [4]:
for images,labels in train_dataloader:
    print(images.shape,labels.shape)
    break

torch.Size([64, 1, 28, 28]) torch.Size([64])


In [6]:
images[0].squeeze().shape

torch.Size([28, 28])

In [15]:
classes = [
    
"T-shirt/top",
"Trouser"
"Pullover",
"Dress",
"Coat",
"Sandal",
"Shirt",
"Sneaker",
"Bag",
"Ankle boot"
]


In [20]:
classes[labels[3].item()]

'Coat'

In [26]:
device = ("cuda" if torch.cuda.is_available()
          else "mps" if torch.mps.is_available()
          else "cpu")
device

'cpu'

In [39]:
class ClothsClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
        nn.Flatten(),
        nn.Linear(28*28, 128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,10)
        )
    def forward(self,x):
        return self.network(x)
    

In [45]:
model = ClothsClassifier().to(device)
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [46]:
epochs = 2
for epoch in range(epochs):
    for batch,(images,labels) in enumerate(train_dataloader):
        images = images.to(device)
        labels = labels.to(device)
        
        pred = model(images)
        loss = loss_fn(pred,labels)
        
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        
        if batch %100==0:
            print(f"Batch: {batch}, Loss: {loss.item()}")

Batch: 0, Loss: 2.294952392578125
Batch: 100, Loss: 0.7799868583679199
Batch: 200, Loss: 0.44511711597442627
Batch: 300, Loss: 0.596030056476593
Batch: 400, Loss: 0.4897603392601013
Batch: 500, Loss: 0.46412980556488037
Batch: 600, Loss: 0.4590095579624176
Batch: 700, Loss: 0.620145857334137
Batch: 800, Loss: 0.5258625149726868
Batch: 900, Loss: 0.504522442817688
Batch: 0, Loss: 0.31225907802581787
Batch: 100, Loss: 0.38682791590690613
Batch: 200, Loss: 0.3138648569583893
Batch: 300, Loss: 0.47531652450561523
Batch: 400, Loss: 0.39254578948020935
Batch: 500, Loss: 0.3694356679916382
Batch: 600, Loss: 0.38257312774658203
Batch: 700, Loss: 0.5467535257339478
Batch: 800, Loss: 0.46436572074890137
Batch: 900, Loss: 0.473090797662735


without momentum 2.31 to 2.03
with momentum 2.31 to 0.69
WITH ADAM 2.31 TO 0.47

In [50]:
# Testing the model
model.eval() # Switch to evaluation mode

all_predicted = []
all_labels = []

with torch.no_grad():
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)

        _,predicted = torch.max(outputs.data, 1)

        # Append Labels and predictions to lists
        all_labels.extend(labels.cpu().numpy())
        all_predicted.extend(predicted.cpu().numpy())

In [52]:
all_labels[:5]

[9, 2, 1, 1, 6]

In [53]:
predicted[:5]

tensor([3, 1, 7, 5, 8])

In [54]:
from sklearn.metrics import classification_report

report = classification_report(all_labels, all_predicted)
print(report)

              precision    recall  f1-score   support

           0       0.80      0.84      0.82      1000
           1       0.98      0.96      0.97      1000
           2       0.86      0.58      0.69      1000
           3       0.84      0.88      0.86      1000
           4       0.63      0.90      0.74      1000
           5       0.96      0.93      0.95      1000
           6       0.73      0.55      0.63      1000
           7       0.88      0.97      0.92      1000
           8       0.93      0.97      0.95      1000
           9       0.97      0.92      0.95      1000

    accuracy                           0.85     10000
   macro avg       0.86      0.85      0.85     10000
weighted avg       0.86      0.85      0.85     10000

